In [1]:
# Penjelasan Konseptual:
# Pada sel ini, kita akan:
# - Mengimpor library yang diperlukan: torch untuk PyTorch, sklearn untuk membuat data dummy, dan matplotlib/numpy untuk analisis tambahan.
# - Membuat dataset dummy menggunakan make_classification dari sklearn.
#   Dataset ini akan memiliki sejumlah fitur tertentu dan dua kelas untuk klasifikasi binari.
# - Setelah itu, data akan di-split menjadi data latih dan data uji.

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Buat dummy data untuk klasifikasi
# n_samples: jumlah data, n_features: jumlah fitur, n_informative: fitur yang benar-benar berpengaruh, n_redundant: fitur duplikat linear.
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, n_classes=2, random_state=42)

# Normalisasi data input agar mempermudah training
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Pisahkan data menjadi train dan test (80%:20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Konversi data menjadi tensor PyTorch
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)


In [2]:
# Penjelasan Konseptual:
# Disini kita membuat sebuah fungsi pembuat model MLP yang dapat mengakomodasi:
# - Jumlah hidden layer
# - Jumlah neuron per layer
# - Jenis fungsi aktivasi
#
# Model sederhana ini akan memiliki:
# Input Layer: sesuai dengan jumlah fitur (misalnya 10)
# Hidden Layer: berdasarkan parameter yang diberikan (bisa 1, 2, atau 3 layer)
# Output Layer: 2 neuron (karena klasifikasi binary)
#
# Fungsi aktivasi yang digunakan dapat dipilih dari: linear (tidak ada aktivasi), sigmoid, relu, tanh.
# Softmax biasanya digunakan di output layer untuk menghasilkan probabilitas, tetapi disini kita juga akan coba di hidden layer hanya untuk demonstrasi.
#
# Catatan:
# - "Linear" di sini berarti tidak menggunakan fungsi aktivasi sama sekali, hanya linear mapping dari input ke output.
# - Bila menggunakan fungsi aktivasi softmax di hidden layer, secara konsep agak tidak lazim, tapi kita coba hanya untuk perbandingan eksperimental.

def create_mlp_model(input_dim, hidden_layers, hidden_neurons, activation_func):
    # Dictionary fungsi aktivasi
    activation_dict = {
        'linear': nn.Identity(),
        'sigmoid': nn.Sigmoid(),
        'relu': nn.ReLU(),
        'tanh': nn.Tanh(),
        # Softmax butuh dim parameter (biasanya dim=1 untuk batch wise)
        # Jika ingin menggunakan di hidden layer, kita asumsikan dim=1.
        'softmax': nn.Softmax(dim=1)
    }

    # List untuk menampung layer MLP secara berurutan
    layers = []

    # Input to first hidden layer
    prev_dim = input_dim
    for i in range(hidden_layers):
        # Tambahkan layer linear
        layers.append(nn.Linear(prev_dim, hidden_neurons))
        # Tambahkan aktivasi
        layers.append(activation_dict[activation_func])
        prev_dim = hidden_neurons

    # Output layer (2 kelas)
    layers.append(nn.Linear(prev_dim, 2))

    # Model sequential
    model = nn.Sequential(*layers)
    return model


In [3]:
# Penjelasan Konseptual:
# Kita perlu fungsi untuk:
# 1. Melatih model (training loop)
# 2. Mengukur akurasi pada data uji
#
# Fungsi train_model akan:
# - Menerima model, data latih, target latih, optimizer, loss function, dan jumlah epoch.
# - Pada setiap epoch, melakukan forward pass, menghitung loss, backward pass (gradient), dan update parameter model.
#
# Fungsi evaluate akan:
# - Menggunakan model terlatih untuk memprediksi kelas data uji.
# - Menghitung akurasi (jumlah prediksi benar / total data uji).

def train_model(model, X_train, y_train, epochs=50, lr=0.001):
    # Optimizer Adam untuk update parameter model
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Loss function Cross Entropy untuk klasifikasi
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        # Set model ke mode training
        model.train()

        optimizer.zero_grad()        # Reset gradient
        outputs = model(X_train)     # Forward pass
        loss = criterion(outputs, y_train)  # Hitung loss
        loss.backward()              # Backward pass
        optimizer.step()             # Update parameter

        # Optional: print loss setiap beberapa epoch untuk monitoring
        if (epoch+1) % 20 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

def evaluate(model, X_test, y_test):
    # Set model ke mode evaluasi
    model.eval()
    # Dengan torch.no_grad() agar tidak menghitung gradient
    with torch.no_grad():
        outputs = model(X_test)
        # Prediksi kelas = argmax dari output logit
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test).sum().item() / len(y_test)
    return accuracy


In [4]:
# Penjelasan Konseptual:
# Kita akan bereksperimen dengan beberapa konfigurasi:
# 1. Jumlah hidden layer: [1, 2, 3]
# 2. Jumlah neuron di hidden layer: [4, 8, 16, 32, 64]
# 3. Fungsi aktivasi: ['linear', 'sigmoid', 'relu', 'tanh', 'softmax']
#
# Kita akan melatih setiap kombinasi secara singkat (misalnya 100 epoch) dan menghitung akurasinya.
# Peringatan: Akan ada banyak kombinasi, mungkin butuh waktu. Sebagai contoh, kita batasi saja sebagian kombinasi atau gunakan epoch lebih sedikit.
#
# Output: Kita akan menyimpan hasil akurasi dalam sebuah list untuk analisis.
#
# Catatan:
# - Dalam praktik, sebaiknya jumlah kombinasi tidak terlalu banyak agar tidak memakan banyak waktu.
# - Disini kita hanya mendemokan konsep.

hidden_layers_list = [1, 2, 3]
hidden_neurons_list = [4, 16, 64]   # Contoh subset saja agar cepat
activation_funcs = ['linear', 'sigmoid', 'relu', 'tanh', 'softmax']

results = []

for hl in hidden_layers_list:
    for hn in hidden_neurons_list:
        for af in activation_funcs:
            print(f"\nTraining model with {hl} hidden layer(s), {hn} neurons, activation={af}")

            # Buat model baru
            model = create_mlp_model(input_dim=10, hidden_layers=hl, hidden_neurons=hn, activation_func=af)

            # Latih model
            train_model(model, X_train_t, y_train_t, epochs=100, lr=0.01)

            # Evaluasi model
            acc = evaluate(model, X_test_t, y_test_t)
            print(f"Accuracy on test: {acc:.4f}")

            # Simpan hasil
            results.append((hl, hn, af, acc))



Training model with 1 hidden layer(s), 4 neurons, activation=linear
Epoch [20/100], Loss: 0.5676
Epoch [40/100], Loss: 0.4291
Epoch [60/100], Loss: 0.4063
Epoch [80/100], Loss: 0.4057
Epoch [100/100], Loss: 0.4055
Accuracy on test: 0.7700

Training model with 1 hidden layer(s), 4 neurons, activation=sigmoid
Epoch [20/100], Loss: 0.6883
Epoch [40/100], Loss: 0.6527
Epoch [60/100], Loss: 0.5804
Epoch [80/100], Loss: 0.4924
Epoch [100/100], Loss: 0.4285
Accuracy on test: 0.7800

Training model with 1 hidden layer(s), 4 neurons, activation=relu
Epoch [20/100], Loss: 0.5595
Epoch [40/100], Loss: 0.3825
Epoch [60/100], Loss: 0.2960
Epoch [80/100], Loss: 0.2591
Epoch [100/100], Loss: 0.2340
Accuracy on test: 0.8600

Training model with 1 hidden layer(s), 4 neurons, activation=tanh
Epoch [20/100], Loss: 0.5803
Epoch [40/100], Loss: 0.4283
Epoch [60/100], Loss: 0.3842
Epoch [80/100], Loss: 0.3382
Epoch [100/100], Loss: 0.3001
Accuracy on test: 0.8000

Training model with 1 hidden layer(s), 4 n

In [5]:
# Penjelasan Konseptual:
# Kita telah mengumpulkan hasil akurasi untuk berbagai kombinasi.
# Sekarang kita bisa melihat kombinasi mana yang menghasilkan akurasi terbaik.
#
# Pada sel ini, kita akan mencetak kembali hasil dengan sorting berdasarkan akurasi.

# Urutkan berdasarkan akurasi descending
sorted_results = sorted(results, key=lambda x: x[3], reverse=True)

print("Top 5 best configurations:")
for i in range(5):
    hl, hn, af, acc = sorted_results[i]
    print(f"Hidden Layers: {hl}, Neurons: {hn}, Activation: {af}, Accuracy: {acc:.4f}")


Top 5 best configurations:
Hidden Layers: 2, Neurons: 64, Activation: tanh, Accuracy: 0.9200
Hidden Layers: 3, Neurons: 64, Activation: tanh, Accuracy: 0.9100
Hidden Layers: 1, Neurons: 64, Activation: tanh, Accuracy: 0.9050
Hidden Layers: 3, Neurons: 16, Activation: relu, Accuracy: 0.9050
Hidden Layers: 1, Neurons: 16, Activation: relu, Accuracy: 0.8950


In [8]:
# Penjelasan Konseptual:
# Pada sel ini, kita akan menyiapkan DataLoader untuk data training karena kita akan membandingkan variasi batch size.
# DataLoader akan mengambil X_train_t dan y_train_t kemudian membuat batch sesuai ukuran batch size yang kita tentukan.
# Kita belum jalankan training di sini; hanya menyiapkan fungsi pembantu terlebih dahulu.

from torch.utils.data import TensorDataset, DataLoader

# Buat dataset PyTorch
train_dataset = TensorDataset(X_train_t, y_train_t)
# DataLoader akan diinisialisasi nanti di dalam loop untuk setiap batch_size.

In [9]:
# Penjelasan Konseptual:
# Fungsi train_model sebelumnya menggunakan keseluruhan data sekaligus.
# Sekarang kita modifikasi agar fungsi tersebut menggunakan DataLoader dengan batch_size yang bervariasi.
#
# Argumen tambahan:
# - batch_size: ukuran batch yang ingin digunakan.
#
# Proses:
# - Set DataLoader dengan batch_size yang diberikan.
# - Pada setiap epoch, kita loop melalui semua batch, melakukan forward, backward, update parameter.
#
# Fungsi evaluate tidak perlu diubah karena evaluasi tetap menggunakan keseluruhan data test secara sekaligus (bisa juga di-batch, tapi tidak masalah untuk data test yang relatif kecil).

def train_model_with_batch(model, X_train, y_train, epochs=50, lr=0.001, batch_size=32):
    # Buat DataLoader untuk training
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Optional: print loss rata-rata per epoch
        if (epoch+1) % 20 == 0:
            avg_loss = running_loss / len(train_loader)
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")


In [10]:
# Penjelasan Konseptual:
# Pada sel ini kita akan melakukan eksperimen komparasi:
# - Epoch: [1, 10, 25, 50, 100, 250]
# - Learning Rate: [10, 1, 0.1, 0.01, 0.001, 0.0001]
# - Batch Size: [16, 32, 64, 128, 256, 512]
#
# Untuk menjaga waktu eksekusi, kita hanya akan mengambil satu konfigurasi arsitektur MLP,
# misalnya: 1 hidden layer, 16 neuron, activation='relu'.
#
# Kita akan iterasi melalui semua kombinasi hyperparameter tersebut. Setiap kombinasi:
# - Buat model baru
# - Latih model dengan parameter tsb
# - Evaluasi model pada test set
# - Simpan hasilnya
#
# Karena jumlah kombinasi besar, ini bisa memakan waktu. Disarankan untuk mencoba subset terlebih dahulu.
# Tetapi disini kita tampilkan kodenya secara lengkap.

epoch_list = [1, 10, 25, 50, 100, 250]
lr_list = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_size_list = [16, 32, 64, 128, 256, 512]

# Untuk eksperimen ini, arsitektur tetap
hidden_layers = 1
hidden_neurons = 16
activation = 'relu'

hyperparam_results = []

for ep in epoch_list:
    for lr in lr_list:
        for bs in batch_size_list:
            print(f"\nTraining model - Epoch: {ep}, LR: {lr}, Batch_Size: {bs}")
            model = create_mlp_model(input_dim=10,
                                     hidden_layers=hidden_layers,
                                     hidden_neurons=hidden_neurons,
                                     activation_func=activation)

            # Train model dengan parameter yang ditentukan
            train_model_with_batch(model, X_train_t, y_train_t, epochs=ep, lr=lr, batch_size=bs)

            # Evaluasi
            acc = evaluate(model, X_test_t, y_test_t)
            print(f"Accuracy on test: {acc:.4f}")

            # Simpan hasil
            hyperparam_results.append((ep, lr, bs, acc))



Training model - Epoch: 1, LR: 10, Batch_Size: 16
Accuracy on test: 0.7550

Training model - Epoch: 1, LR: 10, Batch_Size: 32
Accuracy on test: 0.7550

Training model - Epoch: 1, LR: 10, Batch_Size: 64
Accuracy on test: 0.7350

Training model - Epoch: 1, LR: 10, Batch_Size: 128
Accuracy on test: 0.7550

Training model - Epoch: 1, LR: 10, Batch_Size: 256
Accuracy on test: 0.7700

Training model - Epoch: 1, LR: 10, Batch_Size: 512
Accuracy on test: 0.6000

Training model - Epoch: 1, LR: 1, Batch_Size: 16
Accuracy on test: 0.8100

Training model - Epoch: 1, LR: 1, Batch_Size: 32
Accuracy on test: 0.7650

Training model - Epoch: 1, LR: 1, Batch_Size: 64
Accuracy on test: 0.7650

Training model - Epoch: 1, LR: 1, Batch_Size: 128
Accuracy on test: 0.7750

Training model - Epoch: 1, LR: 1, Batch_Size: 256
Accuracy on test: 0.7100

Training model - Epoch: 1, LR: 1, Batch_Size: 512
Accuracy on test: 0.6450

Training model - Epoch: 1, LR: 0.1, Batch_Size: 16
Accuracy on test: 0.8450

Training m

In [11]:
# Penjelasan Konseptual:
# Setelah semua eksperimen selesai, kita punya hyperparam_results yang berisi tuple (epoch, lr, batch_size, acc).
# Kita akan sortir berdasarkan akurasi untuk melihat konfigurasi mana yang paling baik.

sorted_hyperparam_results = sorted(hyperparam_results, key=lambda x: x[3], reverse=True)

print("Top 5 best configurations based on accuracy:")
for i in range(5):
    ep, lr, bs, acc = sorted_hyperparam_results[i]
    print(f"Epoch: {ep}, LR: {lr}, Batch_Size: {bs}, Accuracy: {acc:.4f}")


Top 5 best configurations based on accuracy:
Epoch: 50, LR: 0.1, Batch_Size: 64, Accuracy: 0.9250
Epoch: 25, LR: 0.1, Batch_Size: 16, Accuracy: 0.9200
Epoch: 100, LR: 0.1, Batch_Size: 32, Accuracy: 0.9200
Epoch: 250, LR: 0.1, Batch_Size: 64, Accuracy: 0.9200
Epoch: 50, LR: 0.01, Batch_Size: 16, Accuracy: 0.9150
